# INS



In [2]:
%matplotlib inline

In [3]:
from __future__ import print_function, division
import numpy as np # matrix manipulations
from matplotlib import pyplot as plt           # this lets you draw inline pictures in the notebooks
import pylab                                   # this allows you to control figure size 
pylab.rcParams['figure.figsize'] = (10.0, 10.0) # this controls figure size in the notebook

In [4]:
from math import sin, cos, atan2, pi, sqrt, asin
from math import radians as deg2rad
from math import degrees as rad2deg

In [6]:
from squaternion import Quaternion, euler2quat, quat2euler, quatNorm

In [3]:
#your python code here
def normalize(x, y, z):
    """Return a unit vector"""
    norm = sqrt(x * x + y * y + z * z)

    # already a unit vector
    if norm == 1.0:
        return (x, y, z)

    if norm > 0.0:
        inorm = 1/norm
        x *= inorm
        y *= inorm
        z *= inorm
    else:
        raise Exception('division by zero: {} {} {}'.format(x, y, z))
    return (x, y, z,)

def skew4(wx, wy, wz):
    return np.array([
        (0, wz, -wy, wx),
        (-wz, 0, wz, -wy),
        (wy, -wx, 0, wz),
        (-wx, -wy, -wz, 0)
    ])


def skew3(wx, wy, wz):
    return np.array([
        (0, -wz, wy),
        (wz, 0, -wx),
        (-wy, wx, 0)
    ])

In [ ]:
def gps2ecef(lat, lon, H):
    # phi = lat
    # lambda = lon
    # H = height above mean sea-level (altitude)
    e = 1.0
    re = 6378137.0  # radius of Earth in meters
    
    # convert degrees to angles
    lat *= pi/180
    lon *= pi/180
    
    rm = re * (1.0 - e**2) / pow(1.0 - e**2 * sin(lat)**2, 3.0 / 2.0)
    rn = re / sqrt(1.0 - e**2 * sin(lat)**2)
    x = (rn + H) * cos(lat) * cos(lon)
    y = (rn + H) * cos(lat) * sin(lon)
    z = (rm + H) * sin(lat)
    return x, y, z

In [6]:
# System equations of motion (eom)
# these follow the Titterton ECEF derivation
def eom(t, X, u):
    """
    State vector
    X = tuple[vx vy vz px py pz qw qx qy qz]
    v - velocity
    p - position
    q - quaternion (orientation)

    These are sensor readings from IMU
    u = [fx fy fz wx wy wz]
    f - force (acceleration)
    w - angular velocity (from gyros)
    """
    q = X[6:]
    f = u[0:3] 
    wx, wy, wz = u[3:]
    p = X[3:6]
    v = X[0:3]
    wie = np.array([0, 0, 7.292115E-15])
    Ceb = np.eye(3)
    W = skew4(wx, wy, wz)
    
    # update local gravity model
    gl = g - np.cross(wie, np.cross(wie, p))
    
    # velocity update
    vd = Ceb.dot(f)-2.0*np.cross(wie, np.cross(wie, v)) + gl
    
    # position update
    pd = v
    
    # orientation update
    qd = 0.5 * W.dot(q)

    # print('vd', vd)
    # print('pd', pd)
    # print('qd', qd)

    XX = np.hstack((vd, pd, qd))
    return XX

In [ ]:
class Sim(object):
    def __init__(self, eom, imu):
        self.eom = eom
        
        # grab the inertial sensors
        self.accels = [x.linear_accel for x in imu]
        self.gyros = [x.angular_vel for x in imu]
        self.mags = [x.magnetic_field for x in imu]
        
        # make a relative delta time from the first time step
        dts = [x.timestamp for x in imu]
        start = dts[0]
        self.dts = [x - start for x in dts]
    
    def run_nav(self):
        """
        These are the navigation only equations, no EKF corrections
        """
        rk = RK(self.eom)
        
        # initial states
        vel = (0,0,0)
        pos = (0,0,0)
        orient = (1,0,0,0)
        X = vel + pos + orient
        
        save = [X]
        
        for i, dt in enumerate(self.dts):
            u = np.array(self.accels[i] + self.gyros[i])
            X = rk.step(X,u,dt)
            save.append(X)